In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import rankdata
%matplotlib inline

In [2]:
data = pd.read_csv('../GenomeCRISPR_full05112017.csv')

/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data.head()

,start,end,chr,strand,pubmed,cellline,condition,sequence,symbol,ensg,log2fc,rc_initial,rc_final,effect,cas,screentype
0,50844073.0,50844096.0,10,+,26472758.0,Jiyoye,viability,GCAGCATCCCAACCAGGTGGAGG,A1CF,ENSG00000148584,0.315907,[260],[244],2.0,hSpCas9,negative selection
1,50814011.0,50814034.0,10,-,26472758.0,Jiyoye,viability,GCGGGAGTGAGAGGACTGGGCGG,A1CF,ENSG00000148584,2.144141,[17],[59],9.0,hSpCas9,negative selection
2,50836111.0,50836134.0,10,+,26472758.0,Jiyoye,viability,ATGACTCTCATACTCCACGAAGG,A1CF,ENSG00000148584,1.426034,[75],[153],8.0,hSpCas9,negative selection
3,50836095.0,50836118.0,10,-,26472758.0,Jiyoye,viability,GAGTCATCGAGCAGCTGCCATGG,A1CF,ENSG00000148584,1.550133,[47],[105],8.0,hSpCas9,negative selection
4,50816234.0,50816257.0,10,-,26472758.0,Jiyoye,viability,AGTCACCCTAGCAAAACCAGTGG,A1CF,ENSG00000148584,0.382513,[58],[57],3.0,hSpCas9,negative selection


In [5]:
# get pmids
pmids = data.pubmed.unique()
pmids

array([26472758., 26627737., 24336569., 24717434., 25494202., 27260157.,
       27013184., 27453484., 24336571., 27383988., 26780180., 27760321.,
       27661255., 27260156., 27869803.,       nan, 28162770.,   999999.,
       28145866., 28700943., 29083409.])

In [9]:
# remove nan and 999999 and meyers pmid
pmids = [26472758., 26627737., 24336569., 24717434., 25494202., 27260157.,
       27013184., 27453484., 24336571., 27383988., 26780180., 27760321.,
       27661255., 27260156., 27869803., 28162770.,
       28145866., 28700943.]
# get meyers pmid
pm1 = 29083409

In [7]:
# group data for meyers
d1 = data[data['pubmed'] == 29083409]
# d1 = d1.groupby(["sequence", "cellline", "symbol"]).median()['effect'].reset_index()

# try removing cellline
d1 = d1.groupby(["sequence", "symbol"]).median()['effect'].reset_index()

In [57]:
# get number of celline intersect
for pm2 in pmids:
    d2 = data[data['pubmed'] == pm2]
    print pm2
    print(len(np.intersect1d(d1.cellline.unique(),d2.cellline.unique())))
    print "################"

26472758.0
0
################
26627737.0
1
################
24336569.0
0
################
24717434.0
0
################
25494202.0
0
################
27260157.0
4
################
27013184.0
0
################
27453484.0
0
################
24336571.0
0
################
27383988.0
0
################
26780180.0
0
################
27760321.0
3
################
27661255.0
0
################
27260156.0
1
################
27869803.0
2
################
28162770.0
9
################
28145866.0
0
################
28700943.0
2
################


In [101]:
# get number of celline intersect
for pm2 in pmids:
    d2 = data[data['pubmed'] == pm2]
    print pm2
    print(len(np.intersect1d(d1.cellline.unique(),d2.cellline.unique())))
    print "################"

26472758.0


AttributeError: 'DataFrame' object has no attribute 'cellline'

In [10]:
# choose just one with highest cellline intersect for now
# pmids= [28162770.0]
for pm2 in pmids:
    d2 = data[data['pubmed'] == pm2]
#     d2 = d2.groupby(["sequence", "cellline", "symbol"]).median()['effect'].reset_index()
    
    # try removing cellline
    d2 = d2.groupby(["sequence", "symbol"]).median()['effect'].reset_index()
    
#     joined = d2.merge(d1, left_on=["sequence", "cellline", "symbol"], right_on=["sequence", "cellline", "symbol"], how="inner")
    
    # try removing cellline
    joined = d2.merge(d1, left_on=["sequence", "symbol"], right_on=["sequence", "symbol"], how="inner")
    
    # do idr in R insteads, so export as csv
    joined.to_csv("joined_" + str(pm2) + ".csv")
    
    
    
    
#     nrows = joined.shape[0]
    
#     # rank effect
# #     eff1 = rankdata(joined['effect_x'])
# #     eff2 = rankdata(joined['effect_y'])
    
#     # try dense method
#     eff1 = rankdata(joined['effect_x'], method="dense")
#     eff2 = rankdata(joined['effect_y'], method="dense")

#     corrGen = np.dot(eff1/float(nrows), eff2/float(nrows))/float(nrows)
#     print pm2
#     print corrGen
    
#     plt.scatter(eff1, eff2, alpha=0.02)
#     plt.show()
    
    print ###################

In [102]:
joined.head()

,sequence,symbol,effect_x,effect_y
0,AAAACACGCAATTTGTGCAGCGG,CXorf58,6.0,0.0
1,AAAACTGTCCATAGTGACCGGGG,NPM3,7.0,-1.0
2,AAAAGAATCAGATAGACCCGTGG,BHMT2,-2.0,-2.0
3,AAACAACGACAGCGAGACCGCGG,SPACA1,6.0,5.0
4,AAACACATCTCTCACCGCGGAGG,GDPD1,2.0,1.0


In [103]:
joined_seq = joined.groupby(["sequence"])

In [108]:
joined_seq.head()

,sequence,symbol,effect_x,effect_y
0,AAAACACGCAATTTGTGCAGCGG,CXorf58,6.0,0.0
1,AAAACTGTCCATAGTGACCGGGG,NPM3,7.0,-1.0
2,AAAAGAATCAGATAGACCCGTGG,BHMT2,-2.0,-2.0
3,AAACAACGACAGCGAGACCGCGG,SPACA1,6.0,5.0
4,AAACACATCTCTCACCGCGGAGG,GDPD1,2.0,1.0
5,AAACTGCTGCTATCTCCCGGCGG,MRPL10,-6.0,-7.0
6,AAACTTCCTGGCTTACACGGCGG,NIPA1,4.0,3.0
7,AAACTTGTGCAAGCAGAAGGAGG,DCN,5.0,4.0
8,AAAGAACCTACAGAGAACTGCGG,MAPK14,0.0,-2.0
9,AAAGAGCACTGGACTCCGGAAGG,SAT1,8.0,6.0


In [26]:
# doench
d2 = data[data['pubmed'] == 26780180]
d2 = d2.groupby(["sequence", "cellline", "symbol"]).median()['effect'].reset_index()